In [113]:
#!pip install bs4
#!pip install requests
#!pip install lxml
#!pip install geocoder

In [3]:
from bs4 import BeautifulSoup as bs
from requests import get
import pandas as pd

In [95]:
response = get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
pagina_web = response.content
sopa = bs(pagina_web)

In [96]:
html_tabla = sopa.find('table', attrs={'class': 'wikitable'})

In [97]:
dfs = pd.read_html(pagina_web)

In [98]:
tabla_municipios = dfs[0]

In [99]:
indexNames = tabla_municipios[ tabla_municipios['Borough'] == 'Not assigned' ].index
tabla_municipios_sin_cosas = tabla_municipios.drop(indexNames)

In [100]:
print(tabla_municipios_sin_cosas.head())
print(tabla_municipios_sin_cosas.shape)

  Postcode           Borough     Neighbourhood
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M6A        North York  Lawrence Heights
6      M6A        North York    Lawrence Manor
(210, 3)


In [101]:
tabla_municipios_sin_cosas.head()
grupos = tabla_municipios_sin_cosas.groupby(['Postcode', 'Borough'],)

salida = grupos['Neighbourhood'].apply(lambda x: ', '.join(list(x)),)


In [102]:
df_agrupado = salida.to_frame().reset_index()
df_agrupado.columns = ['Postcode', 'Borough', 'Neighbourhood']
print(df_agrupado.shape)

(103, 3)


In [105]:
#type(salida)
def asignar_vecindario(row):
    if row['Neighbourhood'] == 'Not assigned':
        return row['Borough']
    else:
        return row['Neighbourhood']
    
columna_nueva = df_agrupado.apply(asignar_vecindario, axis=1)

df_final = df_agrupado.assign(Neighbourhood=columna_nueva.values)


In [109]:
print(df_agrupado[df_agrupado['Neighbourhood'] == 'Not assigned'])
print(df_final[df_final['Neighbourhood'] == 'Not assigned'])
print(df_final[df_final['Postcode'] == 'M9A'])

   Postcode       Borough Neighbourhood
93      M9A  Queen's Park  Not assigned
Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []
   Postcode       Borough Neighbourhood
93      M9A  Queen's Park  Queen's Park


In [110]:
df_final.shape

(103, 3)

In [130]:
csv_coords = pd.read_csv('http://cocl.us/Geospatial_data')



In [159]:
def obtener_coordenadas(postal_code):
    lat_long = csv_coords[csv_coords['Postal Code'] == postal_code].values[0]
    return lat_long[1], lat_long[2]


def obtener_coordenadas_row(row):
    return obtener_coordenadas(row['Postcode'])


In [175]:
lats = []
lons = []
for indice, row in df_final.iterrows():
    postal_code = row['Postcode']
    lat, lon = obtener_coordenadas(postal_code)
    lats.append(lat)
    lons.append(lon)


In [179]:
df_coords = df_final.assign(Latitude=pd.Series(lats), Longitude=pd.Series(lons))

In [180]:
df_coords[df_coords['Postcode'] == 'M5G']

,Postcode,Borough,Neighbourhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [182]:
df_coords.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
